In [32]:
import numpy as np
import matplotlib
import scipy # FrozenLake needs it apparently
import gym

%matplotlib inline

In [33]:
# Initializing and testing the gym environment

env =  gym.make('FrozenLake-v0')
env.reset()

print(env.action_space) 
print(env.observation_space)

for _ in range(1):
    env.render()
    state = env.step(env.action_space.sample())
    print(state)
    
env.env.P[0][0]
env.env.P[5][0]

Discrete(4)
Discrete(16)

SFFF
FHFH
FFFH
HFFG
(1, 0.0, False, {'prob': 0.3333333333333333})


[(1.0, 5, 0, True)]

env.step() returns (next_state, reward, done, probability)<br/>
env.env.P\[state\]\[action\] returns the possible next states which can be achieved <br>

### Action Space
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

# Policy Evaluation
Implementing policy evaluation in Numpy
\begin{equation}
v_{\pi}(s) = \sum_{a}\pi(a|s) \sum_{s\prime}p(s,a,s\prime)\bigg[r(s,a,s\prime) + \gamma v_{\pi}(s\prime) \bigg]
\end{equation}

## Algorithm
1. Input $\pi$, the policy to be evaluated
2. Initialize an array $v(s) = 0$ , for all $s\in S^{+}$
3. Repeat
4. &nbsp;&nbsp;    Initialize $\Delta \gets 0$ 
5. &nbsp;&nbsp;    For each $s \in S$ do:
6. &nbsp;&nbsp;&nbsp;&nbsp;$temp \gets v(s)$
7. &nbsp;&nbsp;&nbsp;&nbsp;$v(s) \gets \sum_{a}\pi(a|s) \sum_{s\prime}p(s,a,s\prime)\bigg[r(s,a,s\prime) + \gamma v(s\prime) \bigg]$
8. &nbsp;&nbsp;&nbsp;&nbsp;$\Delta \gets max(\Delta,|temp - v(s)|)$
9. until $\Delta < \theta$ (a small positive number)
10.Output $v \approx v_{\pi}$

In [65]:
def policy_evaluation(env,policy,gamma = 1.0,theta = 1e-5):
    '''
        policy is a 2D numpy matrix
        policy.shape = (number of states, number of actions)
        gamma = discount factor
        theta = tolerance
        env = env.env
    '''
    v = np.zeros(env.nS) # env.nS = number of states
    complete = False
    while not complete:
        delta = 0
        for s in range(env.nS):
            temp = np.copy(v[s])
            tot_val = 0
            for a in range(env.nA):
                action_val = 0
                transition = env.P[s][a]
                for trans_prob, next_state, reward, done  in transition:
                    action_val += trans_prob*(reward + gamma*v[next_state])    
                tot_val += policy[s,a]*action_val
            v[s] = tot_val
            delta = max(delta, np.abs(temp-v[s]))
        # print(delta)
        if delta < theta:
            complete = True
    
    return v

In [66]:
# Deterministic policy
det_policy = np.array([[0,0,1,0],
                   [0,0,1,0],
                   [0,1,0,0],
                   [1,0,0,0],
                   [0,1,0,0],
                   [1,0,0,0],
                   [0,0,0,1],
                   [0,0,1,0],
                   [0,0,1,0],
                   [0,1,0,0],
                   [1,0,0,0],
                   [1,0,0,0],
                   [0,1,0,0],
                   [0,0,1,0],
                   [0,0,0,1],
                   [0,0,0,1]])

random_policy = np.ones((env.env.nS,env.env.nA))/env.env.nA # all actions equally probable in all states
print(policy_evaluation(env.env,random_policy,0.99,1e-8))

[0.01235611 0.01042444 0.01933842 0.00947774 0.01478704 0.
 0.03889445 0.         0.03260247 0.08433764 0.13781085 0.
 0.         0.17034482 0.43357944 0.        ]


# Policy Iteration
Method for iteratively improving policy
## Algorithm
1. Initialization $v$ and $\pi(s) \in A(s)$ for all $s \in S$
2. Policy evaluation
3. Policy Improvement
4. If Policy is stable go to Step 2

### Policy Improvement
1. $policy_stable \gets true$
2. for each $s \in S$:
3. &nbsp;&nbsp; $temp \gets \pi(s)$
4. &nbsp;&nbsp; $\pi(s) \gets argmax_{a}\sum_{s\prime}p(s\prime|s,a)\bigg[r(s,a,s\prime)+\gamma v(s\prime)\bigg]$
5. If $temp \neq \pi(s)$, then $policy_stable \gets False$

In [67]:
def policy_iteration(env,gamma=1,theta=1e-8):
    '''
        outputs a deterministic policy
    '''
    # count = 100
    policy = np.ones((env.nS,env.nA))/env.nA
    while True:
        
        v = policy_evaluation(env,policy,gamma,theta)
        # policy improvement
        policy_stable = True
        for s in range(env.nS):

            current_action = np.argmax(policy[s])
            action_val = np.zeros(env.nA)
            for a in range(env.nA):
                transition = env.P[s][a]
                for trans_prob, next_state, reward, done in transition:
                    action_val[a] += trans_prob*(reward + gamma*v[next_state])
            best_action = np.argmax(action_val)
            
            if current_action != best_action:
                policy_stable = False
                
            policy[s] = np.eye(env.nA)[best_action]
            
        if policy_stable:
            break
        # count -= 1
            
    return policy,v

In [68]:
policy, v = policy_iteration(env.env,0.99)
print(policy)
print(v)

[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
[0.54202581 0.49880303 0.4706955  0.4568515  0.55845085 0.
 0.35834799 0.         0.59179866 0.64307976 0.6152075  0.
 0.         0.7417204  0.86283741 0.        ]


In [69]:
env.env.P[0][np.argmax(policy[0])]

[(0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 4, 0.0, False)]

In [70]:
for i_episode in range(1):
    observation = env.reset()
    # print(observation)
    for t in range(50):
        env.render()
        print(observation)
        action = np.argmax(policy[observation])
        print(action)
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break


SFFF
FHFH
FFFH
HFFG
0
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
0
  (Left)
SFFF
FHFH
FFFH
HFFG
4
0
  (Left)
SFFF
FHFH
FFFH
HFFG
8
3
  (Up)
SFFF
FHFH
FFFH
HFFG
4
0
  (Left)
SFFF
FHFH
FFFH
HFFG
8
3
  (Up)
SFFF
FHFH
FFFH
HFFG
4
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
0
  (Left)
SFFF
FHFH
FFFH
HFFG
4
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
0
  (Left)
SFFF
FHFH
FFFH
HFFG
4
0
  (Left)
SFFF
FHFH
FFFH
HFFG
8
3
  (Up)
SFFF
FHFH
FFFH
HFFG
8
3
  (Up)
SFFF
FHFH
FFFH
HFFG
9
1
  (Down)
SFFF
FHFH
FFFH
HFFG
13
2
  (Right)
SFFF
FHFH
FFFH
HFFG
9
1
  (Down)
SFFF
FHFH
FFFH
HFFG
10
0
  (Left)
SFFF
FHFH
FFFH
HFFG
6
0
Episode finished after 27 timesteps


# Value Iteration

## Algorithm
1. Repeat
2. &nbsp;&nbsp; $\Delta \gets 0$
3. &nbsp;&nbsp; for each $s \in S$:
4. &nbsp;&nbsp;&nbsp;&nbsp; $temp \gets v(s)$
5. &nbsp;&nbsp;&nbsp;&nbsp; $v(s) \gets max_{a}\sum_{s\prime}p(s\prime | s,a)\bigg[r(s,a,s\prime) + \gamma v(s\prime)\bigg]$
6. &nbsp;&nbsp;&nbsp;&nbsp; $\Delta \gets max(\Delta,|temp - v(s)|)$
7. until $\Delta < \theta$

Output a deterministic policy $\pi$, such that <br>
$\pi(s) = argmax_{a}\sum_{s\prime}p(s\prime|s,a)\bigg[r(s,a,s\prime) + \gamma v(s\prime)\bigg]$

In [75]:
def value_iteration(env,gamma=1,theta=1e-8):
    
    v = np.zeros(env.nS)
    
    while True:
        delta = 0
        for s in range(env.nS):
            temp = np.copy(v[s])
            action_val = np.zeros(env.nA)
            for a in range(env.nA):
                transition = env.P[s][a]
                for trans_prob, next_state, reward, done in transition:
                    action_val[a] += trans_prob*(reward + gamma*v[next_state])
            v[s] = np.max(action_val)
            delta = max(delta,np.abs(temp-v[s]))
        
        if delta < theta:
            break
    
    policy = np.zeros((env.nS,env.nA))
    action_val = np.zeros(env.nA)
    for s in range(env.nS):
        for a in range(env.nA):
            transition = env.P[s][a]
            for trans_prob, next_state, reward, done in transition:
                action_val[a] += trans_prob*(reward + gamma*v[next_state])
        policy[s] = np.eye(env.nA)[np.argmax(action_val)]
    
    return policy,v
            

In [77]:
policy,v = value_iteration(env.env,gamma=0.99)
print(policy)
print(v)

[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
[0.54202581 0.49880303 0.47069551 0.4568515  0.55845085 0.
 0.35834799 0.         0.59179866 0.64307976 0.6152075  0.
 0.         0.7417204  0.86283741 0.        ]


for a discussion on difference between value iteration and policy iteration visit [this](https://stackoverflow.com/questions/37370015/what-is-the-difference-between-value-iteration-and-policy-iteration)